In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# To plot QQ plot
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

## Preprocess the data

# Prediction Generation

1. input data
2. Preprocess Data:
    - Replace -1 with 0s
2. Do something about NA - Create submodels to impute missing values (Random Forest)
2. Fit the model
3. Generate Predictions
4. Generate Confidence Score


In [197]:
prediction_data_1 = pd.read_csv('v2_Inputs_for_Full_Predictions_1_cleaned.csv')

In [198]:
prediction_data_1.shape

(2489558, 31)

In [199]:
prediction_data_1.isna().sum()

Var_1                  0
Var_2                  0
Var_3                  0
Var_4                  0
Var_5_Class_1          0
Var_5_Class_2          0
Var_5_Class_3          0
Var_5_Class_4          0
Var_5_Class_5          0
Var_5_Class_6          0
Var_5_Class_7          0
Var_5_Class_8          0
Var_6_Class_1          0
Var_6_Class_2          0
Var_6_Class_3          0
Var_6_Class_4          0
Var_6_Class_5          0
Var_6_Class_6          0
Var_6_Class_7          0
Var_6_Class_8          0
Var_6_Class_9          0
Var_6_Class_10         0
Var_6_Class_11         0
Var_6_Class_12         0
Var_7                  0
Var_8                  0
Var_9                  0
Var_10                 0
Var_11            604664
Var_12            295215
Var_13                 0
dtype: int64

In [200]:
data_numeric = ['Var_1', 'Var_2', 'Var_3', 'Var_4', 'Var_7', 'Var_8', 'Var_11', 'Var_12', 'Var_13']

In [201]:
# Function to idenfity skewness
def data_skewness(df):
    for i in data_numeric:
        print(i, ":  ", df[i].skew())

In [202]:
data_skewness(prediction_data_1)

Var_1 :   3.9019749056127604
Var_2 :   1.2858898594763082
Var_3 :   18.706620448485587
Var_4 :   -1.2845748252297844
Var_7 :   40.15172939399318
Var_8 :   10.27761779642971
Var_11 :   0.29835446219290374
Var_12 :   0.05928250706545684
Var_13 :   -0.13667133273249232


In [203]:
# Removing skewness. Taking root for positive skewed data and power for negatively skewed data
prediction_data_1["Var_1"] = np.sqrt(prediction_data_1["Var_1"])
prediction_data_1["Var_2"] = np.sqrt(prediction_data_1["Var_2"])
prediction_data_1["Var_3"] = np.cbrt(prediction_data_1["Var_3"])
prediction_data_1["Var_7"] = np.cbrt(prediction_data_1["Var_7"])
prediction_data_1["Var_8"] = np.cbrt(prediction_data_1["Var_8"])
prediction_data_1["Var_11"] = np.power(prediction_data_1["Var_11"], 5)
#prediction_data_1["Var_4"] = np.power(prediction_data_1["Var_4"], 6)

In [204]:
data_skewness(prediction_data_1)

Var_1 :   0.49872883873460444
Var_2 :   0.3576557463036899
Var_3 :   0.9441237516931893
Var_4 :   -1.2845748252297844
Var_7 :   3.992715938235575
Var_8 :   0.4007817435375751
Var_11 :   1.8014263178267473
Var_12 :   0.05928250706545684
Var_13 :   -0.13667133273249232


## Imputing missing data - Var 11
- Create model to predict data.

In [205]:
len(prediction_data_1[prediction_data_1.Var_11.isna() & prediction_data_1.Var_12.isna()]), len(prediction_data_1[prediction_data_1.Var_11.isna()]), len(prediction_data_1[prediction_data_1.Var_12.isna()])

(295215, 604664, 295215)

We notied that whenever Var_12 is missing, Var_11 is also missing. We will 1st build a model to predict Var 11 and hold out Var 12. Then use final model to predict Var_12

In [206]:
#cols = prediction_data_1.columns
cols = ['Var_1', 'Var_2', 'Var_3', 'Var_4', 'Var_5_Class_1', 'Var_5_Class_2',
       'Var_5_Class_3', 'Var_5_Class_4', 'Var_5_Class_5', 'Var_5_Class_6',
       'Var_5_Class_7', 'Var_5_Class_8', 'Var_6_Class_1', 'Var_6_Class_2',
       'Var_6_Class_3', 'Var_6_Class_4', 'Var_6_Class_5', 'Var_6_Class_6',
       'Var_6_Class_7', 'Var_6_Class_8', 'Var_6_Class_9', 'Var_6_Class_10',
       'Var_6_Class_11', 'Var_6_Class_12', 'Var_7', 'Var_8', 'Var_9', 'Var_10','Var_11','Var_13']

In [207]:
impute_independent_var = prediction_data_1[cols]

In [208]:
impute_independent_var.drop(impute_independent_var[pd.isna(impute_independent_var.Var_11)].index, inplace=True)

In [209]:
# Segregating data in independent and dependent variables

X_impute = impute_independent_var.drop(['Var_11'], axis=1)
y_impute = impute_independent_var['Var_11']

In [210]:
from sklearn.model_selection import train_test_split

train_X_impute, test_X_impute, train_y_impute, test_y_impute = train_test_split(X_impute, y_impute, random_state=122, stratify = y_impute)

In [211]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean

In [212]:
from sklearn.ensemble import RandomForestRegressor

var_11_model = RandomForestRegressor(n_estimators=10, random_state=12)

In [213]:
# # define evaluation procedure
# cv_impute = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate model
# scores_impute = cross_val_score(var_11_model, X_impute, y_impute, scoring='accuracy', cv=cv_impute, n_jobs=-1)
# # summarize performance
# print('Accuracy %.3f' % mean(scores_impute))

In [214]:
var_11_model.fit(train_X_impute, train_y_impute)

RandomForestRegressor(n_estimators=10, random_state=12)

In [215]:
impute_independent_var = pd.read_csv('v2_Inputs_for_Full_Predictions_1_cleaned.csv')

In [216]:
X_impute = impute_independent_var.drop(['Var_11', 'Var_12'], axis=1)
y_impute = impute_independent_var['Var_11']

In [217]:
impute_independent_var['Val_11_predicte'] = var_11_model.predict(X_impute)

In [218]:
impute_independent_var[impute_independent_var.Var_11.isnull()]

,Var_1,Var_2,Var_3,Var_4,Var_5_Class_1,Var_5_Class_2,Var_5_Class_3,Var_5_Class_4,Var_5_Class_5,Var_5_Class_6,...,Var_6_Class_11,Var_6_Class_12,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13,Val_11_predicte
22,10,34.628415,0,-18.128415,0,0,0,0,0,0,...,1,0,1,1,0,0,NaN,62.987413,17,-1494.446542
25,8,34.703552,0,-18.203552,0,0,1,0,0,0,...,0,0,1,0,0,0,NaN,NaN,20,-1494.446542
29,10,33.474044,0,-16.974044,0,0,0,0,0,0,...,0,0,1,0,0,0,NaN,NaN,19,-1494.446542
32,11,31.469945,0,-14.969945,0,0,0,0,0,1,...,0,0,1,1,0,0,NaN,NaN,18,-1494.446542
43,12,29.202186,0,-12.702186,0,0,0,0,0,0,...,0,0,1,1,0,0,NaN,NaN,18,-1494.446542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489553,12,0.229508,0,13.650273,0,0,1,0,0,0,...,0,1,3,0,0,0,NaN,NaN,19,-1494.446542
2489554,8,0.247268,0,13.632514,0,0,1,0,0,0,...,0,0,3,0,0,0,NaN,NaN,20,-1494.446542
2489555,12,0.229508,0,13.825137,0,0,1,0,0,0,...,0,1,2,0,0,0,NaN,NaN,19,-1494.446542
2489556,10,0.244536,1,13.497268,0,0,1,0,0,0,...,0,0,2,0,0,0,NaN,NaN,20,-1494.446542


In [219]:
import math

In [220]:
impute_independent_var.Val_11_predicte = impute_independent_var.Val_11_predicte.apply(lambda x: format(x, '.2f'))

In [221]:
impute_independent_var.Var_11.fillna(impute_independent_var.Val_11_predicte, inplace=True)

In [222]:
#impute_independent_var[impute_independent_var.Var_11.isna()].Var_11 = impute_independent_var[impute_independent_var.Var_11.isna()].Var_11.replace(impute_independent_var[impute_independent_var.Var_11.isna()].Var_11, impute_independent_var[impute_independent_var.Var_11.isna()].Val_11_predicte) 

In [134]:
#impute_independent_var.to_csv("Temp_test.csv", index=False)

In [ ]:
# for i in impute_independent_var.Var_11:
#     if(math.isnan(i)):
#         impute_independent_var.Var_11 = impute_independent_var.Val_11_predicte

In [223]:
impute_independent_var.isna().sum()

Var_1                   0
Var_2                   0
Var_3                   0
Var_4                   0
Var_5_Class_1           0
Var_5_Class_2           0
Var_5_Class_3           0
Var_5_Class_4           0
Var_5_Class_5           0
Var_5_Class_6           0
Var_5_Class_7           0
Var_5_Class_8           0
Var_6_Class_1           0
Var_6_Class_2           0
Var_6_Class_3           0
Var_6_Class_4           0
Var_6_Class_5           0
Var_6_Class_6           0
Var_6_Class_7           0
Var_6_Class_8           0
Var_6_Class_9           0
Var_6_Class_10          0
Var_6_Class_11          0
Var_6_Class_12          0
Var_7                   0
Var_8                   0
Var_9                   0
Var_10                  0
Var_11                  0
Var_12             295215
Var_13                  0
Val_11_predicte         0
dtype: int64

In [224]:
impute_independent_var.drop("Val_11_predicte", inplace=True, axis=1)

In [225]:
impute_independent_var.shape, prediction_data_1.shape

((2489558, 31), (2489558, 31))

In [226]:
impute_independent_var.to_csv("Var_11_imputed.csv", index=False)

## Predict Val_12

In [227]:
impute_independent_var = pd.read_csv("Var_11_imputed.csv")

In [228]:
impute_independent_var.shape

(2489558, 31)

In [229]:
data_skewness(impute_independent_var)

Var_1 :   3.9019749056127604
Var_2 :   1.2858898594763082
Var_3 :   18.706620448485587
Var_4 :   -1.2845748252297844
Var_7 :   40.15172939399318
Var_8 :   10.27761779642971
Var_11 :   -1.1923984299476802
Var_12 :   0.05928250706545684
Var_13 :   -0.13667133273249232


In [230]:
# Removing skewness. Taking root for positive skewed data and power for negatively skewed data
impute_independent_var["Var_1"] = np.sqrt(impute_independent_var["Var_1"])
impute_independent_var["Var_2"] = np.sqrt(impute_independent_var["Var_2"])
impute_independent_var["Var_3"] = np.cbrt(impute_independent_var["Var_3"])
impute_independent_var["Var_7"] = np.cbrt(impute_independent_var["Var_7"])
impute_independent_var["Var_8"] = np.cbrt(impute_independent_var["Var_8"])
impute_independent_var["Var_11"] = np.power(impute_independent_var["Var_11"], 5)
#prediction_data_1["Var_4"] = np.power(prediction_data_1["Var_4"], 6)

In [231]:
impute_independent_var.isna().sum()

Var_1                  0
Var_2                  0
Var_3                  0
Var_4                  0
Var_5_Class_1          0
Var_5_Class_2          0
Var_5_Class_3          0
Var_5_Class_4          0
Var_5_Class_5          0
Var_5_Class_6          0
Var_5_Class_7          0
Var_5_Class_8          0
Var_6_Class_1          0
Var_6_Class_2          0
Var_6_Class_3          0
Var_6_Class_4          0
Var_6_Class_5          0
Var_6_Class_6          0
Var_6_Class_7          0
Var_6_Class_8          0
Var_6_Class_9          0
Var_6_Class_10         0
Var_6_Class_11         0
Var_6_Class_12         0
Var_7                  0
Var_8                  0
Var_9                  0
Var_10                 0
Var_11                 0
Var_12            295215
Var_13                 0
dtype: int64

In [232]:
impute_independent_var.drop(impute_independent_var[pd.isna(impute_independent_var.Var_12)].index, inplace=True)

In [233]:
# Segregating data in independent and dependent variables

X_impute = impute_independent_var.drop(['Var_12', 'Var_11'], axis=1)
y_impute = impute_independent_var['Var_12']

In [234]:
from sklearn.model_selection import train_test_split

train_X_impute, test_X_impute, train_y_impute, test_y_impute = train_test_split(X_impute, y_impute, random_state=122, stratify = y_impute)

In [235]:
from sklearn.ensemble import RandomForestRegressor

var_12_model = RandomForestRegressor(n_estimators=10, random_state=12)

In [236]:
train_X_impute.isna().sum()

Var_1             0
Var_2             0
Var_3             0
Var_4             0
Var_5_Class_1     0
Var_5_Class_2     0
Var_5_Class_3     0
Var_5_Class_4     0
Var_5_Class_5     0
Var_5_Class_6     0
Var_5_Class_7     0
Var_5_Class_8     0
Var_6_Class_1     0
Var_6_Class_2     0
Var_6_Class_3     0
Var_6_Class_4     0
Var_6_Class_5     0
Var_6_Class_6     0
Var_6_Class_7     0
Var_6_Class_8     0
Var_6_Class_9     0
Var_6_Class_10    0
Var_6_Class_11    0
Var_6_Class_12    0
Var_7             0
Var_8             0
Var_9             0
Var_10            0
Var_13            0
dtype: int64

In [237]:
var_12_model.fit(train_X_impute, train_y_impute)

RandomForestRegressor(n_estimators=10, random_state=12)

In [238]:
impute_independent_var = pd.read_csv('Var_11_imputed.csv')

X_impute = impute_independent_var.drop(['Var_11', 'Var_12'], axis=1)
y_impute = impute_independent_var['Var_12']

In [239]:
impute_independent_var['Val_12_predicte'] = var_12_model.predict(X_impute)

In [240]:
impute_independent_var[impute_independent_var.Var_12.isnull()]

,Var_1,Var_2,Var_3,Var_4,Var_5_Class_1,Var_5_Class_2,Var_5_Class_3,Var_5_Class_4,Var_5_Class_5,Var_5_Class_6,...,Var_6_Class_11,Var_6_Class_12,Var_7,Var_8,Var_9,Var_10,Var_11,Var_12,Var_13,Val_12_predicte
25,8,34.703552,0,-18.203552,0,0,1,0,0,0,...,0,0,1,0,0,0,-1494.45,NaN,20,62.987413
29,10,33.474044,0,-16.974044,0,0,0,0,0,0,...,0,0,1,0,0,0,-1494.45,NaN,19,62.987413
32,11,31.469945,0,-14.969945,0,0,0,0,0,1,...,0,0,1,1,0,0,-1494.45,NaN,18,62.987413
43,12,29.202186,0,-12.702186,0,0,0,0,0,0,...,0,0,1,1,0,0,-1494.45,NaN,18,62.987413
51,19,27.908470,1,-11.408470,0,0,1,0,0,0,...,0,0,1,0,0,0,-1494.45,NaN,18,62.987413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489553,12,0.229508,0,13.650273,0,0,1,0,0,0,...,0,1,3,0,0,0,-1494.45,NaN,19,62.987413
2489554,8,0.247268,0,13.632514,0,0,1,0,0,0,...,0,0,3,0,0,0,-1494.45,NaN,20,62.987413
2489555,12,0.229508,0,13.825137,0,0,1,0,0,0,...,0,1,2,0,0,0,-1494.45,NaN,19,62.987413
2489556,10,0.244536,1,13.497268,0,0,1,0,0,0,...,0,0,2,0,0,0,-1494.45,NaN,20,62.987413


In [241]:
impute_independent_var.Val_12_predicte = impute_independent_var.Val_12_predicte.apply(lambda x: format(x, '.2f'))

In [242]:
impute_independent_var.Var_12.isna().sum()

295215

In [243]:
impute_independent_var.Var_12.fillna(impute_independent_var.Val_12_predicte, inplace=True)

In [ ]:
#impute_independent_var[impute_independent_var.Var_12.isna()]['Var_12'] = impute_independent_var[impute_independent_var.Var_12.isna()]['Var_12'].replace(impute_independent_var.Var_12, impute_independent_var.Val_12_predicte) 

In [ ]:
# for i in impute_independent_var.Var_12:
#     if(math.isnan(i)):
#         impute_independent_var.Var_12 = impute_independent_var.Val_12_predicte

In [244]:
impute_independent_var.drop("Val_12_predicte", inplace=True, axis=1)

In [245]:
impute_independent_var.isna().sum()

Var_1             0
Var_2             0
Var_3             0
Var_4             0
Var_5_Class_1     0
Var_5_Class_2     0
Var_5_Class_3     0
Var_5_Class_4     0
Var_5_Class_5     0
Var_5_Class_6     0
Var_5_Class_7     0
Var_5_Class_8     0
Var_6_Class_1     0
Var_6_Class_2     0
Var_6_Class_3     0
Var_6_Class_4     0
Var_6_Class_5     0
Var_6_Class_6     0
Var_6_Class_7     0
Var_6_Class_8     0
Var_6_Class_9     0
Var_6_Class_10    0
Var_6_Class_11    0
Var_6_Class_12    0
Var_7             0
Var_8             0
Var_9             0
Var_10            0
Var_11            0
Var_12            0
Var_13            0
dtype: int64

In [246]:
impute_independent_var.to_csv("v2_Inputs_for_Full_Predictions_1_imputed.csv", index=False)